In [4]:
!pip uninstall typing_extensions -y
!pip install --upgrade typing_extensions


!pip install torch torchvision


Found existing installation: typing_extensions 4.12.2
Uninstalling typing_extensions-4.12.2:
  Successfully uninstalled typing_extensions-4.12.2
Defaulting to user installation because normal site-packages is not writeable
  Using cached typing_extensions-4.14.0-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.14.0-py3-none-any.whl (43 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17

In [ ]:
import torch
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from torchvision.utils import make_grid
import torchvision.transforms as T
from scipy.ndimage import zoom
from PIL import Image, ImageDraw, ImageFont

# Función para cargar y obtener cortes de una imagen NIfTI
def load_nifti_image(nifti_path):
    img = nib.load(nifti_path).get_fdata()
    return img

def get_slices(img):
    """Obtiene cortes Axial, Coronal y Sagital y los rota 90° antihorario."""
    # Verificar las dimensiones de la imagen
    if len(img.shape) == 4:  # Si es 4D, tomar el primer volumen
        img = img[..., 0]
    elif len(img.shape) != 3:  # Si no es 3D, lanzar un error
        raise ValueError(f"Se esperaba una imagen 3D, pero se obtuvo una con forma {img.shape}")

    # Extraer las dimensiones
    z, y, x = img.shape

    # Obtener los cortes
    axial = np.rot90(img[z // 2, :, :])      # Vista axial
    coronal = np.rot90(img[:, y // 2, :])    # Vista coronal
    sagittal = np.rot90(img[:, :, x // 2])   # Vista sagital

    return axial, coronal, sagittal

def normalize_image(image):
    """ Normaliza la imagen entre 0 y 1 """
    return (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-8)  # Evita división por cero

from torchvision import transforms

def resize_image(image, final_size=(256, 256)):
    """
    Redimensiona la imagen preservando la relación de aspecto y luego aplica padding para alcanzar el tamaño final.
    """
    pil_img = T.ToPILImage()(image)

    # Obtener el tamaño original
    w, h = pil_img.size  

    # Calcular la escala manteniendo la proporción
    scale = min(final_size[0] / w, final_size[1] / h)  
    new_w, new_h = int(w * scale), int(h * scale)  

    # Redimensionar con la escala correcta
    resized = pil_img.resize((new_w, new_h), Image.BILINEAR)

    # Calcular padding necesario
    pad_left = (final_size[0] - new_w) // 2
    pad_top = (final_size[1] - new_h) // 2
    pad_right = final_size[0] - new_w - pad_left
    pad_bottom = final_size[1] - new_h - pad_top

    # Aplicar padding
    transform_pad = transforms.Pad((pad_left, pad_top, pad_right, pad_bottom), fill=0)
    padded = transform_pad(T.ToTensor()(resized))

    return padded.numpy().squeeze()  # Convertimos de tensor a array




def nifti_to_tensor(nifti_path, target_size=(256, 256)):
    """
    Carga un archivo NIfTI y lo convierte en un tensor normalizado,
    asegurando que todas las imágenes tengan el mismo tamaño.
    """
    img = load_nifti_image(nifti_path)
    axial, coronal, sagittal = get_slices(img)

    # Normalizar imágenes
    axial = normalize_image(axial)
    coronal = normalize_image(coronal)
    sagittal = normalize_image(sagittal)

    # Redimensionar todas las imágenes al mismo tamaño
    axial = resize_image(axial, target_size)
    coronal = resize_image(coronal, target_size)
    sagittal = resize_image(sagittal, target_size)

    # Convertir a tensores de PyTorch (1 canal)
    transform = T.Compose([T.ToTensor(), T.Normalize(mean=[0.5], std=[0.5])])
    axial_t = transform(axial).unsqueeze(0)
    coronal_t = transform(coronal).unsqueeze(0)
    sagittal_t = transform(sagittal).unsqueeze(0)

    return torch.cat([axial_t, coronal_t, sagittal_t], dim=0)

def add_labels(image, subject_id):
    """
    Agrega etiquetas específicas para cada fila de preprocesamiento directamente sobre cada fila en la cuadrícula.
    """
    draw = ImageDraw.Draw(image)

    # Intentar cargar una fuente TrueType
    try:
        font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"  # Ruta común en sistemas Linux
        font = ImageFont.truetype(font_path, 30)
    except Exception as e:
        print(f"Error al cargar la fuente TrueType: {e}")
        font = ImageFont.load_default()  # Fuente por defecto si falla

    W, H = image.size
    n_rows = 5  # Número de filas en la cuadrícula
    row_height = H // n_rows  # Altura de cada fila

    # Etiquetas para cada fila
    labels = ["Original", "ANTS Rigid", "ANTS Affine", "FLIRT Rigid", "FLIRT Affine"]

    # Posicionar las etiquetas sobre cada fila
    for i, label in enumerate(labels):
        y_position = i * row_height + 10  # Posición vertical sobre cada fila
        text_width = font.getsize(label)[0]
        draw.text(((W - text_width) // 2, y_position), label, fill="red", font=font)  # Centrado horizontalmente

    # Agregar título centrado arriba
    title = f"{subject_id} Preprocessing Comparison"
    text_width = font.getsize(title)[0]
    draw.text(((W - text_width) // 2, 5), title, fill="black", font=font)

    return image

def plot_brain_grid(subject_id, paths, output_dir):
    """
    Genera y guarda la imagen comparativa de la imagen original y los 4 preprocesamientos.
    """
    # Cargar y procesar las imágenes
    brain_slices = [nifti_to_tensor(p) for p in paths]
    grid = make_grid(torch.cat(brain_slices, dim=0), nrow=3, padding=2, normalize=True)
    grid_np = grid.numpy().transpose(1, 2, 0)

    # Convertir a imagen de PIL y agregar etiquetas
    img = Image.fromarray((grid_np * 255).astype(np.uint8))
    labeled_img = add_labels(img, subject_id)

    # Guardar la imagen
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{subject_id}_comparison.jpg")
    labeled_img.save(output_path)

# Directorios de entrada y salida
input_dir_original = "/data/Lautaro/Documentos/BrainAgeCOVID/DATOS/Raw_T1/datos_check_de_preprocesamiento"
input_dir_preproc1 = "/data/Lautaro/Documentos/BrainAgeCOVID/DATOS/Raw_T1/datos_check_de_preprocesamiento/preprocessed_ANTS_rigid"
input_dir_preproc2 = "/data/Lautaro/Documentos/BrainAgeCOVID/DATOS/Raw_T1/datos_check_de_preprocesamiento/preprocessed_ANTS_affine"
input_dir_preproc3 = "/data/Lautaro/Documentos/BrainAgeCOVID/DATOS/Raw_T1/datos_check_de_preprocesamiento/preprocessed_FLIRT_rigid"
input_dir_preproc4 = "/data/Lautaro/Documentos/BrainAgeCOVID/DATOS/Raw_T1/datos_check_de_preprocesamiento/preprocessed_FLIRT_affine"
output_dir = "./jpeg_preprocessing_analysis_comparison"

# Obtener la lista de sujetos a procesar basado en el nombre del archivo (sin extensión) de la carpeta "original"
subject_ids = [os.path.splitext(os.path.splitext(f)[0])[0]
               for f in os.listdir(input_dir_original) if f.endswith(".nii.gz")]

# Generar imágenes para cada sujeto
for subject_id in subject_ids:
    original_path = os.path.join(input_dir_original, f"{subject_id}.nii.gz")
    preproc1_path = os.path.join(input_dir_preproc1, f"{subject_id}.nii.gz")
    preproc2_path = os.path.join(input_dir_preproc2, f"{subject_id}.nii.gz")
    preproc3_path = os.path.join(input_dir_preproc3, f"{subject_id}.nii.gz")
    preproc4_path = os.path.join(input_dir_preproc4, f"{subject_id}.nii.gz")

    if all(os.path.exists(p) for p in [original_path, preproc1_path, preproc2_path, preproc3_path, preproc4_path]):
        plot_brain_grid(subject_id, [original_path, preproc1_path, preproc2_path, preproc3_path, preproc4_path], output_dir)
        print(f"Imagen guardada: {subject_id}_comparison.jpg")
    else:
        print(f"Archivos no encontrados para {subject_id}: {original_path}, {preproc1_path}, {preproc2_path}, {preproc3_path}, o {preproc4_path}")


Imagen guardada: IXI002-Guys-0828-T1_comparison.jpg
Imagen guardada: IXI012-HH-1211-T1_comparison.jpg
Imagen guardada: IXI013-HH-1212-T1_comparison.jpg
Imagen guardada: IXI014-HH-1236-T1_comparison.jpg
Imagen guardada: IXI015-HH-1258-T1_comparison.jpg
